# Generating the null model for comparison to modification data

In this file, we will generate a null model consisting of random residues, rather than modifiable ones, meant to represent the impact splicing would have on residues if they are randomly selected from the proteome. To do so, we will first perform the mapping and projection process for all residues in the protoeme, and then randomly select a subset of these residues to define as a "modifiable". With this null model, we can then calculate constitutive rates and altered flank rates for each residue, and compare these to the rates observed in the modification data.

## Table of Contents

1. [Load Data](#load-data)
2. [Mapping and Projection of Residues to the Genome](#construct-null-residue-dataframes)
    1. [Mapping Residues to the Genome](#map-residues-to-genome)
    2. [Projection of Residues to the Genome](#project-residues-to-alternative-transcripts)
3. [Generate null model distributions of constitutive and altered flank rates](#generate-null-model-distributions)
    1. [Constitutive rates](#generate-null-model-distributions-for-constitutive-rates)
    2. [Altered flank rates](#generate-null-model-distributions-for-altered-flank-rates)



## Load Data

In [ ]:
from ExonPTMapper import mapping, config
import os
import pandas as pd
import tqdm 
import numpy as np

#store figshare directory
figshare_dir = 'C:/Users/Sam/OneDrive/Documents/GradSchool/Research/Splicing/Paper_Prep/PTM_Splicing_FigShare/'

#load mapping data
mapper = mapping.PTM_mapper()

## Construct null residue dataframes

In [ ]:
protein_list = []
ptm_list = []
pos_list = []
residue_list = []
genes = []
transcripts = []
modifications = []

#reduce translator to only the Uniprot and transcript IDs
trim_translator = config.translator[['Transcript stable ID', 'UniProtKB/Swiss-Prot ID']].drop_duplicates()

#for all transcripts with matching information in the mapper, get the amino acid sequence
for transcript_id in tqdm.tqdm(config.available_transcripts):
    #get transcript info
    trans_info = mapper.transcripts.loc[transcript_id]
    protein = config.translator.loc[config.translator['Transcript stable ID'] == transcript_id, 'UniProtKB/Swiss-Prot ID']
    
    #iterate through transcript sequence and add each residue to the lists, using same structure as residue_info dataframe
    seq = trans_info['Amino Acid Sequence']
    for pos, residue in zip(range(1,len(seq)+1),seq):
        protein_list.append(protein.values[0])
        ptm_list.append(protein+residue+str(pos))
        pos_list.append(pos)
        residue_list.append(residue)
        genes.append(trans_info['Gene stable ID'])
        transcripts.append(trans_info.name)
        modifications.append('Null')
        
result = pd.DataFrame({'Gene': genes, 'Protein':protein_list,'Transcript': transcripts, 'Residue': residue_list, 
              'PTM Location (AA)':pos_list, 'Modification': 'No Mod'})
result.to_csv(figshare_dir + '/Analysis_For_Paper/Null_Model/residue_info.csv', index = False)

## Map Residues to Genome 

Using the same mapping/projection procedure as with the true ptm data, we can map residues to their location in the genome and project them to alternative residues. Only difference is that we will not be collapsing the residue information into a single row (duplicates for the same residue if found in multiple canonical transcripts). This is a result of memory limitations but does not change the analysis.

### Get location of residue in transcript

In [ ]:
#create copy of existing residue_info dataframe
residue_info = residue_info.rename({'Transcript':'Transcripts'}, axis = 1)
#separate residue_info dataframe by unique transcripts
#residue_info['Transcripts'] = residue_info['Transcripts'].apply(lambda x: x.split(';'))
#residue_info = residue_info.explode('Transcripts')

print('Getting location of PTMs in exon, transcript, and gene')

#extract transcript level info required for mapping process (coding start and location of splice boundaries)
transcript_data = mapper.transcripts[['Relative CDS Start (bp)', 'Exon cuts']].copy()
#remove transcripts without necessary data (nan, or with error string)
transcript_data = transcript_data.dropna(subset = ['Exon cuts', 'Relative CDS Start (bp)'])

#convert exon cuts into list containing integers rather than a single string
transcript_data['Exon cuts'] = transcript_data['Exon cuts'].apply(lambda cut: np.array([int(x) for x in cut.split(',')]))
transcript_data = transcript_data.dropna(subset = ['Exon cuts', 'Relative CDS Start (bp)'])

#add transcript data to ptm information
residue_info = residue_info.merge(transcript_data, left_on = 'Transcripts', right_index = True, how = 'left')
residue_info = residue_info.dropna(subset = 'Exon cuts')

#get transcript location of PTMs
residue_info['Transcript Location (NC)'] = ((residue_info['PTM Location (AA)']-1)*3 + residue_info['Relative CDS Start (bp)'])

#get rank of exon in transcript, based on transcript location and exon cuts. To do so, find the first exon cut which is greater than transcript location.
min_exon_rank = mapper.exons.groupby('Transcript stable ID')['Exon rank in transcript'].min()
exon_rank = []
n_dist_list = []
c_dist_list = []
min_dist_list = []
ragged_list = []
for i, row in tqdm.tqdm(residue_info.iterrows(), total = residue_info.shape[0], desc = 'Identify PTM-containing exons'):
    #get minimum exon rank in transcript, for rare case when first exons don't have sequence info

    #get distance of PTM from splice boundaries by subtracting PTM location in transcript by splice boundaries
    normed_cuts = row['Transcript Location (NC)'] - row['Exon cuts']
    #find the first negative number in normed_cuts (larger than transcript loc), which will indicate the correct exon rank
    for c in range(len(normed_cuts)):
        if normed_cuts[c] <  0:
            #add 1, as currently in pythonic coordinates (if missing first exon ranks, add additional)
            exon_rank.append(c+min_exon_rank[row['Transcripts']])

            #record distance n-terminal boundary/start of exon (if exon rank is 1, this will be the same as the transcript location, else use the normed cuts)
            if c == 0:
                n_distance = row['Transcript Location (NC)']
            else:
                n_distance = normed_cuts[c-1]
            
            #record distance to c-terminal boundary/end of exon
            c_distance = row['Exon cuts'][c] - (row['Transcript Location (NC)'] + 3)

            #record the minimum distance to boundary
            min_distance = min([n_distance, c_distance])

            #assess if ptm is ragged (coded for by two exons, found at splice boundary)
            ragged = min_distance < 0

            #save data to lists
            n_dist_list.append(n_distance)
            c_dist_list.append(c_distance)
            min_dist_list.append(min_distance)
            ragged_list.append(ragged)
            break

#save data to columns in dataframe
residue_info['Exon rank in transcript'] = exon_rank
residue_info['Distance to N-terminal Splice Boundary (NC)'] = n_dist_list
residue_info['Distance to C-terminal Splice Boundary (NC)'] = c_dist_list
residue_info['Distance to Closest Boundary (NC)'] = min_dist_list
residue_info['Ragged'] = ragged_list


residue_info.to_csv(figshare_dir + '/Analysis_For_Paper/Null_Model/residue_info.csv', index = False)


### Get location of residue in genome

In [ ]:
#remove exon cuts column, no longer needed
residue_info = residue_info.drop('Exon cuts', axis = 1)

#add exon level information required for rest of mapping process (mapping to genomic location)
exon_info = mapper.exons[['Transcript stable ID', 'Exon stable ID', 'Exon rank in transcript', 'Exon Start (Gene)', 'Exon End (Gene)', 'Exon Start (Protein)', 'Exon End (Protein)', 'Warnings (AA Seq)']].copy()
residue_info = residue_info.merge(exon_info, left_on = ['Transcripts', 'Exon rank in transcript'], right_on = ['Transcript stable ID', 'Exon rank in transcript'], how = 'left')


#add gene info to ptm dataframe (which strand and chromosome ptm is located)
residue_info = residue_info.merge(mapper.genes[['Chromosome/scaffold name', 'Strand']], left_on = 'Genes', right_index = True)

#get genomic locatio of ptms and coordinates
gene_loc = []
coordinates = []
second_exon = []
ragged_loc_list = []
for i, row in tqdm(residue_info.iterrows(), total = residue_info.shape[0], desc = 'Getting location of PTMs in genome'):
    #check strand of gene, calculate location of ptm in gene based on which strand
    if row['Strand'] == 1:  
        loc = row['Distance to N-terminal Splice Boundary (NC)'] + row['Exon Start (Gene)']
    else:
        loc = row['Exon End (Gene)'] - row['Distance to N-terminal Splice Boundary (NC)']

    #check if able to get location, if so convert to integer
    if loc == loc:
        loc = int(loc)
    gene_loc.append(loc)
        
    #given gene location, get genomic coordinates in the following format (<chromosome>:<start>-<stop>:<strand>)
    if not row['Ragged']:   #genomic coordinates when gene is not found at splice boundary
        coordinates.append(mapping.getGenomicCoordinates(row['Chromosome/scaffold name'], loc, row['Strand']))
        second_exon.append(np.nan)
        ragged_loc_list.append(np.nan)
    else:   #genomic coordinates when gene is found at splice boundary (ragged site)
        #identify the other exon contributing to ragged PTM site, get start of this exon 
        next_rank = row['Exon rank in transcript'] + 1
        transcript = mapper.exons[mapper.exons['Transcript stable ID'] == row['Transcripts']]
        next_exon = transcript[transcript['Exon rank in transcript'] == next_rank].squeeze()

        #get location of ragged ptm to feed into function
        ragged_loc = int(next_exon['Exon Start (Gene)'] if row['Strand'] == 1 else next_exon['Exon End (Gene)'])
        min_dist = row['Distance to Closest Boundary (NC)']
        #get coordinates of ragged PTM, save exon id of second contributing exon
        coordinates.append(mapping.getRaggedCoordinates(row['Chromosome/scaffold name'], loc, ragged_loc, min_dist, row['Strand']))
        ragged_loc_list.append(ragged_loc)
        second_exon.append(next_exon['Exon stable ID'])

#save data to ptm dataframe
residue_info['Gene Location (NC)'] = gene_loc
residue_info['Genomic Coordinates'] = coordinates
residue_info['Second Contributing Exon'] = second_exon
residue_info['Ragged Genomic Location'] = ragged_loc_list
residue_info = residue_info.drop(['Exon Start (Gene)', 'Exon End (Gene)'], axis = 1)
        

### Get location of residue in the exon, based on amino acid coordiantes

In [ ]:
#get ptm location in exon in amino acid coordinates
exon_aa_loc = []
for i, row in tqdm(residue_info.iterrows(), total = residue_info.shape[0], desc = 'Getting residue number within each exon'):
    if row['Warnings (AA Seq)'] == row['Warnings (AA Seq)']:
        exon_aa_loc.append(np.nan)
    else:
        loc = round(row['PTM Location (AA)'] - float(row['Exon Start (Protein)']), 2)
        exon_aa_loc.append(loc)
        
residue_info['Exon Location (AA)'] = exon_aa_loc

#add column with ptm name (<UniProtID>_<Residue><Position>)
residue_info['PTM'] = residue_info['Protein'] + '_' + residue_info['Residue']+ residue_info['PTM Location (AA)'].astype(str)

### Construct residue coordinates dataframe (collapse residue into location in genome)

In [ ]:
print('Constructing residue coordinates dataframe')
#save new dataframe which will be trimmed version of ptm info with each row containing a PTM mapped to unique genomic coordinates
residue_coordinates = residue_info[['Genomic Coordinates', 'PTM','Residue', 'Modification', 'Chromosome/scaffold name', 'Strand','Gene Location (NC)', 'Ragged', 'Ragged Genomic Location', 'Exon stable ID']].copy()
residue_coordinates = residue_coordinates.dropna(subset = 'Gene Location (NC)')
residue_coordinates = residue_coordinates.drop_duplicates()
residue_coordinates = residue_coordinates.astype({'Gene Location (NC)': int, 'Strand':int, 'Ragged':bool})

#group modifications for the same ptm in the same row
grouped = residue_coordinates.groupby(['Genomic Coordinates', 'Chromosome/scaffold name', 'Residue', 'Strand', 'Gene Location (NC)', 'Ragged'])
residue_coordinates = pd.concat([grouped['PTM'].agg(lambda x: ';'.join(np.unique(x))), grouped['Ragged Genomic Location'].apply(lambda x: np.unique(x)[0]), grouped['Modification'].agg(lambda x: ';'.join(np.unique(x))), grouped['Exon stable ID'].agg(lambda x: ';'.join(np.unique(x)))], axis = 1)
residue_coordinates = residue_coordinates.reset_index()
residue_coordinates = residue_coordinates.rename({'PTM':'Source of PTM', 'Exon stable ID': 'Source Exons'}, axis = 1)

#make genomic coordinates the index of dataframe
residue_coordinates.index = residue_coordinates['Genomic Coordinates']
residue_coordinates = residue_coordinates.drop('Genomic Coordinates', axis = 1)
residue_coordinates = residue_coordinates.drop_duplicates()

Saved the residue coordinates dataframe to modification specific files to reduce memory load of running the projection onto genomic locations.

In [ ]:
#dictionary for converting from single letter to full names of amino acids
residue_dict = {'A': 'Alanine', 'C': 'Cysteine', 'D':'AsparticAcid','E':'GlutamicAcid',
                'F':'Phenylalanine','G':'Glycine','H':'Histidine','I':'Isoleucine', 
                'K':'Lysine', 'L':'Leucine','M':'Methionine', 'N':'Asparagine',
                'P':'Proline','Q':'Glutamine','R':'Arginine','S':'Serine', 'T':'Threonine',
                'V':'Valine','W':'Tryptophan','Y':'Tyrosine'}

for residue in residue_dict.keys():
    subset = residue_coordinates[residue_coordinates['Residue'] == residue]
    subset.to_csv(figshare_dir + f'/Analysis_For_Paper/Null_Model/residue_coordinates/{residue_dict[residue]}_coordinates.csv')

## Project residues to alternative transcripts

In [ ]:
mods = ['Serine','Threonine', 'Tyrosine', 'Phenylalanine', 'Glycine', 'Histidine', 'Isoleucine', 'Leucine', 'Proline', 'Glutamine', 'Valine', 'Tryptophan', 'Alanine', 'Cysteine', 'AsparticAcid', 'GlutamicAcid', 'Lysine', 'Methionine', 'Asparagine', 'Arginine']
residues = ['S','T', 'Y', 'F', 'G', 'H', 'I', 'L', 'P', 'Q', 'V', 'W', 'A', 'C', 'D', 'E', 'K', 'M', 'N', 'R']

#iterate through each residue and project ptms separately
for mod, residue in zip(mods, residues):
  if not os.path.exists(figshare_dir + f'/Analysis_For_Paper/Null_Model/residue_coordinates/{mod}.csv'):
    #load residue coordinates and info into mapper object
    mapper.ptm_coordinates = pd.read_csv(figshare_dir + f'/Analysis_For_Paper/Null_Model/residue_coordinates/{mod}.csv', index_col = 0, dtype = {'Source of PTM': str, 'Chromosome/scaffold name': str, 'Gene Location':int,
                                                  'Ragged': str})
    mapper.ptm_info = residue_info[residue_info['Residue'] == residue]
    mapper.alternative_ptms = None
    
    #project residues to alternative exons as would do for real data
    mapper.projectPTMs_toAlternativeExons()

    #save project residues to save directory
    if not os.path.exists(figshare_dir + f'/Analysis_For_Paper/Null_Model/projected_residues'):
      os.mkdir(figshare_dir + f'/Analysis_For_Paper/Null_Model/projected_residues')
    mapper.alternative_ptms.to_csv(figshare_dir + f'/Analysis_For_Paper/Null_Model/projected_residues/{mod}.csv')

## Generate null model distributions

To generate the null model distributions, we follow the same procedure as the true modification data, performing the following steps for each modification class:
1. Sample a random subset of residues, matching the same number of residues for that modification.
2. Extract the mapping and projection data for those residues, treating them as "modified".
3. Calculate the constitutive rate and altered flank rate as done with true modifications.
4. Save the calculated rates to a list. Repeat a total of 100 times.
5. Save rates to a  text file.


### Load Data

In [ ]:
def getIsoformSpecificPTMs(alternative_ptms, isoforms, required_length = 20):
    """
    Reduce alternative ptm dataframe to ptms that are unique to a specific protein sequence, rather than a specific transcript. This avoids issue in which multiple transcripts can code for the same protein isoform.

    Parameters
    ----------
    required_length : int, optional
        Minimum length of protein isoform to be considered. The default is 20 amino acids.

    Returns
    -------
    isoform_ptms (as attribute of mapper object): dataframe
        Dataframe of PTMs that are unique to a specific protein isoform, rather than a specific transcript.
    """
    #get isoform data, then separate isoform data into unique rows for each transcript
    isoforms = isoforms[isoforms['Isoform Type'] == 'Alternative'].copy()
    isoforms = isoforms[['Isoform ID', 'Transcript stable ID', 'Isoform Length']]
    isoforms['Transcript stable ID'] = isoforms['Transcript stable ID'].apply(lambda x: x.split(';'))
    isoforms = isoforms.explode('Transcript stable ID')
    isoforms = isoforms.rename({'Transcript stable ID': 'Alternative Transcript'}, axis = 1)

    #merge isoform and alternative ptm information
    isoform_ptms = alternative_ptms.merge(isoforms, on = 'Alternative Transcript', how = 'left').copy()


    #drop duplicate rows by isoform id
    isoform_ptms = isoform_ptms.drop_duplicates(subset = ['Isoform ID', 'Source of PTM'])
    isoform_ptms = isoform_ptms.drop('Alternative Transcript', axis = 1)

    #drop isoforms shorter than 20 amino acids
    isoform_ptms = isoform_ptms[isoform_ptms['Isoform Length'] >= required_length]
    return isoform_ptms

In [ ]:
#dictionary for converting between single letter and full names of amino acids
residue_dict = {'A': 'Alanine', 'C': 'Cysteine', 'D':'AsparticAcid','E':'GlutamicAcid',
                'F':'Phenylalanine','G':'Glycine','H':'Histidine','I':'Isoleucine', 
                'K':'Lysine', 'L':'Leucine','M':'Methionine', 'N':'Asparagine',
                'P':'Proline','Q':'Glutamine','R':'Arginine','S':'Serine', 'T':'Threonine',
                'V':'Valine','W':'Tryptophan','Y':'Tyrosine'}

#load real ptm data
mapper = mapping.PTM_mapper()
mapper.getAllFlankingSeqs(flank_size = 10)
#get isoform specific ptms (collapse to )
mapper.isoform_ptms = pd.read_csv(figshare_dir + '/PTM_Data/processed_data_dir/isoform_ptms.csv', index_col = 0)
#calculate real ptm conservation
mapper.calculate_PTMconservation()
#copy ptm info
ptm_info = mapper.ptm_info.copy()

#get file for converting modification types to their grouped classess (phosphotyrosine to phosphorylation, etc.)
mod_groups = pd.read_csv(figshare_dir + '/External_Data/modification_conversion.csv', header = 0)


#separate based on modifications, such that each row is a unique residue/modification type pair
exploded_mods = ptm_info.copy()
exploded_mods["Modification"] = exploded_mods['Modification'].apply(lambda x: x.split(';'))
exploded_mods = exploded_mods.explode('Modification').reset_index()
exploded_mods = exploded_mods.rename({'index':'PTM'}, axis = 1)
exploded_mods = exploded_mods.drop_duplicates()
exploded_mods = exploded_mods.merge(mod_groups[['Mod Name', 'Mod Class']], left_on = 'Modification', right_on = 'Mod Name')
exploded_mods = exploded_mods.drop(['Mod Name', 'Modification'], axis = 1)
exploded_mods = exploded_mods.drop_duplicates()

#load residue information
residue_info = pd.read_csv(figshare_dir + '/Analysis_For_Paper/Null_Model/residue_info.csv')
residue_info['PTM'] = residue_info.index

### Generate Null Distribution for Constitutive Rates

In [ ]:
mods_to_test = ['PHOS','UBIQ', 'ACET', 'GLCN', 'METH','SUMO', 'DIMETH', 'NTRY', 'HYDR', 'TRIMETH','SULF', 'PALM', 'CITR', 'GGLU']
#mods_to_test = ['HYDR', 'TRIMETH','SULF', 'PALM', 'GGLU']
#mods_to_test = ['CITR']
mod_column = 'Mod Class'
for mod in mods_to_test:
    print(f'Getting null {mod} data')
    tmp_mods = exploded_mods[exploded_mods[mod_column] == mod].copy()
    #load toy mapped residues
    residues = tmp_mods['Residue'].unique()
    sizes = tmp_mods.groupby('Residue').size()
    mapped_residues = {}
    for res in residues:
        try:
            mapped_residues[res] = getIsoformSpecificPTMs(pd.read_csv(f'../MockModifications/mapped/{residue_dict[res]}.csv', dtype = {'Exon ID (Alternative)':str, 
                                                                        'Chromosome/scaffold name': str, 'Ragged':str, 'Genomic Coordinates':str, 'Exon ID (Canonical)': str, 
                                                                            'Alternative Residue': str, 'Protein':str, 'Canonical Protein Position (AA)': str, 
                                                                        'Alternative Protein Position (AA)':str,'Mapping Result':str, 'Second Exon': str}), isoforms)
        except:
            print(f'{residue_dict[res]} not found. {sizes[res]} with modification.')
            
    unique_res = tmp_mods['Residue'].unique()
    num_mod = tmp_mods.groupby('Residue').size()[mapped_residues.keys()]
    
    np.random.seed(100)
    rate_list = []
    for i in tqdm.tqdm(range(100)):
        #for each residue, randomly sample the number that matches that found within modification population
        sampled_results = None
        for residue in num_mod.index:
            residue_list = residue_info.loc[residue_info['Residue'] == residue, "PTM"].unique()
            sample = np.random.choice(residue_list, size = num_mod[residue], replace = False)
            if sampled_results is None:
                conservation_data = residue_info.loc[sample].copy()
                sampled_results = mapped_residues[residue][mapped_residues[residue]['Source of PTM'].isin(sample)]
            else:
                conservation_data = pd.concat([conservation_data, residue_info.loc[sample]])
                sampled_results = pd.concat([sampled_results,mapped_residues[residue][mapped_residues[residue]['Source of PTM'].isin(sample)]])

        conserved_transcripts = sampled_results[sampled_results['Mapping Result'] == 'Success'].groupby('Source of PTM')['Isoform ID'].apply(list)
        num_conserved = conserved_transcripts.apply(len)
        num_conserved.name = 'Conserved'

        lost_transcripts = sampled_results[sampled_results['Mapping Result'] != 'Success'].groupby('Source of PTM')['Isoform ID'].apply(list)
        num_lost = lost_transcripts.apply(len)
        num_lost.name = 'Lost'
                                               

        conservation_data['Number of Conserved Transcripts'] = num_conserved
        conservation_data['Number of Lost Transcripts'] = num_lost                                    

        conservation_score = []
        for ptm in conservation_data.index:
            num_conserved = conservation_data.loc[ptm, 'Number of Conserved Transcripts']
            num_lost = conservation_data.loc[ptm, 'Number of Lost Transcripts']
            #check if there are any conserved transcripts (or if not and is NaN)
            if num_conserved != num_conserved and num_lost == num_lost:
                conservation_score.append(0)
            elif num_conserved != num_conserved and num_lost != num_lost:
                conservation_score.append(1)
            #check if any lost transcripts: if not replace NaN with 0 when calculating
            elif num_lost != num_lost and num_conserved == num_conserved:
                conservation_score.append(1)
            else:
                conservation_score.append(num_conserved/(num_conserved+num_lost))

        conservation_data['Score'] = conservation_score
        conservation_data['Constitutive'] = (conservation_data['Score'] == 1)*1
        rate = conservation_data[conservation_data['Constitutive'] == 1].shape[0]/conservation_data.shape[0]
        rate_list.append(rate)
        
    # open file
    with open(figshare_dir + f'/Analysis_For_Paper/Null_Model/Null_Constitutive_Rates/{mod}.txt', 'w+') as f:

        # write elements of list
        for items in rate_list:
            f.write('%s\n' %items)

        print("File written successfully")


    # close the file
    f.close()

### Get Null Model Distributions for Altered Flank Rates

In [ ]:
import re

def getFlankingSequences(isoforms, isoform_ptms, flank_size = 5):
    flank_seq = []
    for i, row in isoform_ptms.iterrows():
        iso_id = row['Isoform ID']
        protein_sequence = isoforms.loc[iso_id]
        if isinstance(protein_sequence, pd.Series):
            protein_sequence = protein_sequence['Amino Acid Sequence']
                #if multiple transcripts associated with protein, only use first transcript (should be same seq)
            if row['Alternative Protein Position (AA)'] == row['Alternative Protein Position (AA)']:
                pos = int(float(row['Alternative Protein Position (AA)']))
                if pos <= flank_size:
                    #if amino acid does not have long enough N-terminal flanking sequence, add spaces to cushion sequence
                    spaces = ''.join([' ' for i in range(flank_size - pos + 1)])
                    flank_seq.append(spaces + protein_sequence[0:pos-1]+protein_sequence[pos-1].lower()+protein_sequence[pos:pos+flank_size])
                elif len(protein_sequence)-pos <= flank_size:
                    #if amino acid does not have long enough C-terminal flanking sequence, add spaces to cushion sequence
                    spaces = ''.join([' ' for i in range(flank_size - (len(protein_sequence)-pos))])
                    flank_seq.append(protein_sequence[pos-flank_size-1:pos-1]+protein_sequence[pos-1].lower()+protein_sequence[pos:]+spaces)
                else:
                    #full flanking sequence available
                    flank_seq.append(protein_sequence[pos-flank_size-1:pos-1]+protein_sequence[pos-1].lower()+protein_sequence[pos:pos+flank_size])
            else:
                flank_seq.append(np.nan)
        else:
            flank_seq.append(np.nan)
    return flank_seq


def getFlankingSequences_can(transcripts, residue_info, flank_size = 5):
    flank_seq = []
    for i, row in residue_info.iterrows():
        trans_id = row['Transcripts']
        if ';' in trans_id:
            trans_id = trans_id.split(';')[0]
            
        protein_sequence = transcripts.loc[trans_id]
        if isinstance(protein_sequence, pd.Series):
            protein_sequence = protein_sequence['Amino Acid Sequence']
                #if multiple transcripts associated with protein, only use first transcript (should be same seq)
            if row['PTM Location (AA)'] == row['PTM Location (AA)']:
                pos = int(float(row['PTM Location (AA)']))
                if pos <= flank_size:
                    #if amino acid does not have long enough N-terminal flanking sequence, add spaces to cushion sequence
                    spaces = ''.join([' ' for i in range(flank_size - pos + 1)])
                    flank_seq.append(spaces + protein_sequence[0:pos-1]+protein_sequence[pos-1].lower()+protein_sequence[pos:pos+flank_size])
                elif len(protein_sequence)-pos <= flank_size:
                    #if amino acid does not have long enough C-terminal flanking sequence, add spaces to cushion sequence
                    spaces = ''.join([' ' for i in range(flank_size - (len(protein_sequence)-pos))])
                    flank_seq.append(protein_sequence[pos-flank_size-1:pos-1]+protein_sequence[pos-1].lower()+protein_sequence[pos:]+spaces)
                else:
                    #full flanking sequence available
                    flank_seq.append(protein_sequence[pos-flank_size-1:pos-1]+protein_sequence[pos-1].lower()+protein_sequence[pos:pos+flank_size])
            else:
                flank_seq.append(np.nan)
        else:
            flank_seq.append(np.nan)
    return flank_seq

def matchedFlankSeq(seq1, seq2):
    return (seq1 == seq2)*1

def compareAllFlankSeqs(residue_info, res, flank_size = 5, unique_isoforms = True):
    """
    Given the alternative ptms and canonical ptm data, compare flanking sequences and determine if they are identical. 
    
    Parameters
    ----------
    flank_size:
        size of the flanking sequence to compare. IMPORTANT, this should not be larger than the available flanking sequence in the ptm_info dataframe
    """
    conserved_flank = []
    for i in res.index:
        #check if alt flanking seq exists
        alt_flank = res.loc[i, 'Flanking Sequence']
        if alt_flank != alt_flank:
            conserved_flank.append(np.nan)
        else:
            
            ptm = res.loc[i, 'Source of PTM']
            if ';' in ptm:
                for p in ptm.split(';'):
                    try:
                        tmp_info = residue_info.loc[p]
                        if isinstance(tmp_info, pd.DataFrame):
                            iso_id = res.loc[i, 'Isoform ID']
                            trans_id = isoforms.loc[iso_id, 'Transcript stable ID'].split(';')
                            tmp_info = tmp_info[tmp_info['Transcripts'] == trans_id[0]]
                            if tmp_info.shape[0] == 0:
                                conserved_flank.append(np.nan)
                            else:
                                can_flank = tmp_info['Flanking Sequence'].values[0]
                            can_flank = tmp_info['Flanking Sequence'].values[0]
                        else:
                            can_flank = tmp_info['Flanking Sequence']
                        #can_flank = can_flank[n_term:c_term]
                        conserved_flank.append(matchedFlankSeq(can_flank, alt_flank))
                        success = True
                    except:
                        pass
                if not success:
                    conserved_flank.append(matchedFlankSeq(can_flank, alt_flank))
            else:
                
                tmp_info = residue_info.loc[ptm]
                if isinstance(tmp_info, pd.DataFrame):
                    iso_id = res.loc[i, 'Isoform ID']
                    trans_id = isoforms.loc[iso_id, 'Transcript stable ID'].split(';')
                    tmp_info = tmp_info[tmp_info['Transcripts'] == trans_id[0]]
                    if tmp_info.shape[0] == 0:
                        conserved_flank.append(np.nan)
                    else:
                        can_flank = tmp_info['Flanking Sequence'].values[0]
                else:
                    can_flank = tmp_info['Flanking Sequence']
                #can_flank = can_flank[n_term:c_term]
                conserved_flank.append(matchedFlankSeq(can_flank, alt_flank))
                
                    #conserved_flank.append(np.nan)
    return conserved_flank

def compareAllFlankSeqs2(residue_info, res, flank_size = 5, unique_isoforms = True):
    """
    Given the alternative ptms and canonical ptm data, compare flanking sequences and determine if they are identical. 
    
    Parameters
    ----------
    flank_size:
        size of the flanking sequence to compare. IMPORTANT, this should not be larger than the available flanking sequence in the ptm_info dataframe
    """
    conserved_flank = []
    for i in res.index:
        #check if alt flanking seq exists
        alt_flank = res.loc[i, 'Flanking Sequence']
        if alt_flank != alt_flank:
            conserved_flank.append(np.nan)
        else:
            
            ptm = res.loc[i, 'Source of PTM'].split(';')[0]
            try:
                tmp_info = residue_info.loc[ptm]
                can_flank = tmp_info['Flanking Sequence']
                #can_flank = can_flank[n_term:c_term]
                conserved_flank.append(matchedFlankSeq(can_flank, alt_flank))
                success = True
            except Exception as e:
                print(e)
                conserved_flank.append(np.nan)
                
                    #conserved_flank.append(np.nan)

    return conserved_flank

In [ ]:
transcripts = pd.read_csv(config.processed_data_dir + 'transcripts.csv', index_col = 0)
residue_info['Flanking Sequence'] = getFlankingSequences_can(transcripts, residue_info, flank_size = 5)

mods_to_test = ['PHOS','UBIQ', 'ACET', 'GLCN', 'METH','SUMO', 'DIMETH', 'NTRY', 'HYDR', 'TRIMETH','SULF', 'PALM', 'GGLU']
mod_column = 'Mod Class'
for mod in mods_to_test:
    print(f'Getting null {mod} data')
    tmp_mods = exploded_mods[exploded_mods[mod_column] == mod].copy()
    #load toy mapped residues
    residues = tmp_mods['Residue'].unique()
    sizes = tmp_mods.groupby('Residue').size()
    mapped_residues = {}
    for res in residues:
        try:
            mapped_residues[res] = getIsoformSpecificPTMs(pd.read_csv(f'../MockModifications/mapped/{residue_dict[res]}.csv', dtype = {'Exon ID (Alternative)':str, 
                                                                        'Chromosome/scaffold name': str, 'Ragged':str, 'Genomic Coordinates':str, 'Exon ID (Canonical)': str, 
                                                                            'Alternative Residue': str, 'Protein':str, 'Canonical Protein Position (AA)': str, 
                                                                        'Alternative Protein Position (AA)':str,'Mapping Result':str, 'Second Exon':str}), isoforms)
        except:
            print(f'{residue_dict[res]} not found. {sizes[res]} with modification.')
            
    unique_res = tmp_mods['Residue'].unique()
    num_mod = tmp_mods.groupby('Residue').size()[mapped_residues.keys()]
    
    np.random.seed(100)
    flank_conservation_list = []
    for i in tqdm.tqdm(range(100)):
        #for each residue, randomly sample the number that matches that found within modification population
        sampled_results = None
        for residue in num_mod.index:
            #residue_list = residue_info.loc[residue_info['Residue'] == residue, "PTM"].unique()
            residue_list = np.unique(residue_info[residue_info['Residue'] == residue].index.values)
            sample = np.random.choice(residue_list, size = num_mod[residue], replace = False)
            if sampled_results is None:
                conservation_data = residue_info.loc[sample].copy()
                sampled_results = mapped_residues[residue][mapped_residues[residue]['Source of PTM'].isin(sample)]
            else:
                conservation_data = pd.concat([conservation_data, residue_info.loc[sample]])
                sampled_results = pd.concat([sampled_results,mapped_residues[residue][mapped_residues[residue]['Source of PTM'].isin(sample)]])

        #get flanking sequences
        sampled_results = sampled_results.reset_index()
        sampled_results['Flanking Sequence'] = getFlankingSequences(mapper.isoforms, sampled_results)
        sampled_results = sampled_results.dropna(subset = 'Flanking Sequence')
        conserved_flanks = compareAllFlankSeqs2(residue_info, sampled_results)
        conserved_flanks = [f for f in conserved_flanks if f == f]
        #calculate fraction of ptms with conserved flank
        flank_conservation_list.append(1- sum(conserved_flanks)/len(conserved_flanks))
        
                                               
        
    # open file
    with open(figshare_dir + f'/Analysis_For_Paper/Null_Model/Null_Flank_Rates/{mod}.txt', 'w+') as f:

        # write elements of list
        for items in flank_conservation_list:
            f.write('%s\n' %items)

        print("File written successfully")


    # close the file
    f.close()